In [1]:
import pyodbc as PDB  # Drive para conexão com banco de dados
import pandas  as pd
import sys
import os
import traceback
import json
import re

In [2]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

In [3]:
datasource = pd.read_excel(r'D:\Ondrive\OneDrive - Implanta Informatica\Migração arquivos Anexos\ClientesComESemComfiguracaoArquivosAnexos.xlsx',sheet_name='registros')

In [4]:
datasource.query("Categoria in('cau','core','coren','crea','cra','cro','crf','confere','crn','crt','oab','crefito','crmv','crq','crp')")['Categoria'].unique()

array(['cau', 'cra', 'cro', 'crq', 'crt', 'confere', 'core', 'coren',
       'crea', 'crefito', 'crf', 'crmv', 'crn', 'crp', 'oab'],
      dtype=object)

In [5]:
filter_categorias = datasource.query("Categoria in('cau','core','coren','crea','cra','cro','crf','confere','crn','crt','oab','crefito','crmv','crq','crp') and  UtilizaAzure =='NAO'")[['Cliente','Categoria','MigracaoDiaria','UtilizaAzure','StorageAccount','ContainerAzureStorageArquivosAnexos','CaminhoCacheLocalAzureStorageArquivosAnexos','ConnectionStringAzureStorageArquivosAnexos','HorarioMigracaoArquivosAnexosParaAzureStorage']]

In [6]:
server = 'rgprd-sqlsrv-prd01.database.windows.net'
database = 'cau-ac.implanta.net.br'
username = 'wesley.neves@implantainformatica.com.br'
password = 'enzo_96086512'
driver= '{ODBC Driver 17 for SQL Server}'

In [8]:
def get_conection(driver,server,database,username,password):
    _conexao = PDB.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password +';Authentication=ActiveDirectoryPassword')
    return _conexao

In [9]:
sql_query_Validation_configuracao = (
" SELECT X.* "
"  FROM( "
"          SELECT DB_NAME() AS Cliente,"
"                 ("
"                     SELECT COUNT(1)FROM Sistema.ArquivosAnexos"
"                 ) AS TotalRegistros,"
"                 ("
"                     SELECT COUNT(1)"
"                       FROM Sistema.ArquivosAnexos"
"                      WHERE"
"                         ConteudoEmStorageExterno = 0"
"                 ) AS TotalArquivosBanco,"
"                 ("
"                     SELECT C.Valor"
"                       FROM Sistema.Configuracoes AS C"
"                      WHERE"
"                         C.Configuracao = 'UsaAzureStorageArquivosAnexos'"
"                 ) AS UsaAzureStorageArquivosAnexos,"
"                 ("
"                     SELECT C.Valor"
"                       FROM Sistema.Configuracoes AS C"
"                      WHERE"
"                         C.Configuracao = 'ContainerAzureStorageArquivosAnexos'"
"                 ) AS ContainerAzureStorageArquivosAnexos,"
"                 ("
"                     SELECT C.Valor"
"                       FROM Sistema.Configuracoes AS C"
"                      WHERE"
"                         C.Configuracao = 'HorarioMigracaoArquivosAnexosParaAzureStorage'"
"                 ) AS HorarioMigracaoArquivosAnexosParaAzureStorage,"
"                 ("
"                     SELECT C.Valor"
"                       FROM Sistema.Configuracoes AS C"
"                      WHERE"
"                         C.Configuracao = 'QuantidadeDeArquivosMigradosPorExecucao'"
"                 ) AS QuantidadeDeArquivosMigradosPorExecucao"
"      )X(Cliente, TotalRegistros, TotalArquivosBanco, UsaAzureStorageArquivosAnexos, "
" ContainerAzureStorageArquivosAnexos,HorarioMigracaoArquivosAnexosParaAzureStorage, QuantidadeDeArquivosMigradosPorExecucao);"

)

## controle de migração

In [10]:
pathFIleControleMigracao =r'D:\Ondrive\OneDrive - Implanta Informatica\Migração arquivos Anexos\ControleMigracao.xlsx'

dataFrame_Controle = pd.DataFrame()

In [11]:
import os.path
from datetime import date
import numpy as np

In [12]:

arquivoExiste = os.path.isfile(pathFIleControleMigracao)

# if(arquivoExiste ==False):
pathFIleControleMigracaodataFrame_Controle = pd.DataFrame(columns=['Cliente','TotalArquivosBanco','UsaAzureStorageArquivosAnexos','ContainerAzureStorageArquivosAnexos','HorarioMigracaoArquivosAnexosParaAzureStorage','QuantidadeDeArquivosMigradosPorExecucao','Dia'])

# else:
#     dataFrame_Controle = pd.read_excel(pathFIle,sheet_name='registros')


In [13]:
colunas =dataFrame_Controle.columns

appended_data = []


In [16]:
filter_categorias.shape


(233, 9)

In [17]:
for index , row in filter_categorias.iterrows():
    database = row["Cliente"]
    categoria = row["Categoria"]
    migracaoDiaria = row["MigracaoDiaria"]
    UtilizaAzure = row["UtilizaAzure"]
    
    storageAccount =  row["StorageAccount"]
    container =row["ContainerAzureStorageArquivosAnexos"]
    horarioMigracao =row["HorarioMigracaoArquivosAnexosParaAzureStorage"]
    connect_str_to_storage = row["ConnectionStringAzureStorageArquivosAnexos"]
    cLocalAzureStorageArquivosAnexos =row["CaminhoCacheLocalAzureStorageArquivosAnexos"]

    
    
    blob_service_client = BlobServiceClient.from_connection_string(connect_str_to_storage)
    existContainer = blob_service_client.get_container_client(container).exists()

    if(existContainer ==False):
        print('Container não configurado:',container)

    if(existContainer ==True):
        print(database)
        _con = get_conection(driver,server,database,username,password)
        cursor = _con.cursor()

        df = pd.read_sql(sql_query_Validation_configuracao, _con)
        
        appended_data.append(df)
        

        # df = pd.DataFrame(dados,columns=colunas)

        _con.commit()
        cursor.close()


appended_data = pd.concat(appended_data)

cau-al.implanta.net.br
cau-am.implanta.net.br
cau-ap.implanta.net.br
cau-ba.implanta.net.br
cau-br.implanta.net.br
cau-ce.implanta.net.br
cau-df.implanta.net.br
cau-es.implanta.net.br
cau-go.implanta.net.br
cau-ma.implanta.net.br
cau-mg.implanta.net.br
cau-ms.implanta.net.br
cau-mt.implanta.net.br
cau-pa.implanta.net.br
cau-pb.implanta.net.br
cau-pe.implanta.net.br
cau-pi.implanta.net.br
cau-pr.implanta.net.br
cau-rn.implanta.net.br
cau-ro.implanta.net.br
cau-rr.implanta.net.br
cau-rs.implanta.net.br
cau-sc.implanta.net.br
cau-se.implanta.net.br
cau-to.implanta.net.br
cra-ac.implanta.net.br
cro-rs.implanta.net.br
crq-ba.implanta.net.br
crq-ce.implanta.net.br
crq-es.implanta.net.br
crq-go.implanta.net.br
crq-ma.implanta.net.br
crq-ms.implanta.net.br
crq-pa.implanta.net.br
crq-pb.implanta.net.br
crq-pe.implanta.net.br
crq-pi.implanta.net.br
crq-pr.implanta.net.br
crq-rj.implanta.net.br
crq-rn.implanta.net.br
crq-rs.implanta.net.br
crq-sc.implanta.net.br
crq-se.implanta.net.br
crq-sp.impl

,Cliente,TotalRegistros,TotalArquivosBanco,UsaAzureStorageArquivosAnexos,ContainerAzureStorageArquivosAnexos,HorarioMigracaoArquivosAnexosParaAzureStorage,QuantidadeDeArquivosMigradosPorExecucao
0,cau-al.implanta.net.br,2971,2033,true,cau-al,23:00:00,500
0,cau-am.implanta.net.br,3819,1931,true,cau-am,21:00:00,500
0,cau-ap.implanta.net.br,4018,2641,true,cau-ap,23:00:00,500
0,cau-ba.implanta.net.br,1284,836,true,cau-ba,22:00:00,500
0,cau-br.implanta.net.br,78043,76897,true,cau-br,01:00:00,500
...,...,...,...,...,...,...,...
0,crea-rs.implanta.net.br,1158,118,true,crea-rs,23:00:00,500
0,crea-sc.implanta.net.br,862,0,true,crea-sc,01:00:00,500
0,crea-se.implanta.net.br,1164,162,true,crea-se,22:00:00,500
0,crea-sp.implanta.net.br,5262,4758,true,crea-sp,19:00:00,500


## criação de novas variaveis

In [18]:
def CalculaQuantidadeDiasRestantes(totalArquivosNoBanco,quantidadeAMigrarDiarimente):
    return int(round((int(totalArquivosNoBanco) / int(quantidadeAMigrarDiarimente)),0))


In [20]:
def AplicaFinalizacaoDeMigracao(totalArquivosNoBanco,quantidadeAMigrarDiarimente):
    return 'SIM' if int(totalArquivosNoBanco) < 50 else 'NAO'

In [21]:
appended_data["dia"] = date.today().strftime("%d/%m/%Y")

appended_data['QuantidadeDiasRestantes'] = appended_data.apply(lambda x: CalculaQuantidadeDiasRestantes(x['TotalArquivosBanco'], x['QuantidadeDeArquivosMigradosPorExecucao']), axis=1)
appended_data['Finalizou'] = appended_data.apply(lambda x: AplicaFinalizacaoDeMigracao(x['TotalArquivosBanco'], x['QuantidadeDeArquivosMigradosPorExecucao']), axis=1)



In [50]:
appended_data

,Cliente,TotalRegistros,TotalArquivosBanco,UsaAzureStorageArquivosAnexos,ContainerAzureStorageArquivosAnexos,HorarioMigracaoArquivosAnexosParaAzureStorage,QuantidadeDeArquivosMigradosPorExecucao,dia,QuantidadeDiasRestantes,Finalizou
0,cau-al.implanta.net.br,2971,1099,true,cau-al,23:00:00,500,07/05/2021,2,NAO
0,cau-am.implanta.net.br,3823,987,true,cau-am,21:00:00,500,07/05/2021,2,NAO
0,cau-ap.implanta.net.br,4024,2182,true,cau-ap,23:00:00,500,07/05/2021,4,NAO
0,cau-ba.implanta.net.br,1309,52,true,cau-ba,22:00:00,500,07/05/2021,0,SIM
0,cau-br.implanta.net.br,78195,75947,true,cau-br,01:00:00,500,07/05/2021,152,NAO
...,...,...,...,...,...,...,...,...,...,...
0,crea-rs.implanta.net.br,1193,0,true,crea-rs,23:00:00,500,07/05/2021,0,SIM
0,crea-sc.implanta.net.br,869,0,true,crea-sc,01:00:00,500,07/05/2021,0,SIM
0,crea-se.implanta.net.br,1164,0,true,crea-se,22:00:00,500,07/05/2021,0,SIM
0,crea-sp.implanta.net.br,5274,3758,true,crea-sp,19:00:00,500,07/05/2021,8,NAO


## Pandas Tranpose

In [22]:
transpose = appended_data[['Cliente','TotalRegistros','TotalArquivosBanco','QuantidadeDeArquivosMigradosPorExecucao','dia','QuantidadeDiasRestantes','Finalizou']]

transpose.transpose()

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cliente,cau-al.implanta.net.br,cau-am.implanta.net.br,cau-ap.implanta.net.br,cau-ba.implanta.net.br,cau-br.implanta.net.br,cau-ce.implanta.net.br,cau-df.implanta.net.br,cau-es.implanta.net.br,cau-go.implanta.net.br,cau-ma.implanta.net.br,...,oab-ap.implanta.net.br,oab-ce.implanta.net.br,oab-df.implanta.net.br,oab-ms.implanta.net.br,oab-pa.implanta.net.br,oab-pi.implanta.net.br,oab-rn.implanta.net.br,oab-rr.implanta.net.br,oab-rs.implanta.net.br,oab-se.implanta.net.br
TotalRegistros,2977,3823,4078,1309,78820,3589,3343,3664,4307,3656,...,1,2195,4765,17,2230,43,17,2,68104,152
TotalArquivosBanco,33,28,346,52,73591,32,3,556,74,26,...,1,0,2245,0,0,0,0,2,65692,0
QuantidadeDeArquivosMigradosPorExecucao,500,500,500,500,500,500,500,500,500,500,...,500,500,500,500,500,500,500,500,500,500
dia,12/05/2021,12/05/2021,12/05/2021,12/05/2021,12/05/2021,12/05/2021,12/05/2021,12/05/2021,12/05/2021,12/05/2021,...,12/05/2021,12/05/2021,12/05/2021,12/05/2021,12/05/2021,12/05/2021,12/05/2021,12/05/2021,12/05/2021,12/05/2021
QuantidadeDiasRestantes,0,0,1,0,147,0,0,1,0,0,...,0,0,4,0,0,0,0,0,131,0
Finalizou,SIM,SIM,NAO,NAO,NAO,SIM,SIM,NAO,NAO,SIM,...,SIM,SIM,NAO,SIM,SIM,SIM,SIM,SIM,NAO,SIM


## Pandas Pivot

In [23]:


pivotdata = pd.pivot_table(transpose, columns=['QuantidadeDeArquivosMigradosPorExecucao'], index=['Cliente', 'TotalRegistros','dia', 'TotalArquivosBanco','QuantidadeDiasRestantes','Finalizou'],aggfunc=np.sum)

## Clientes Finalizados

In [25]:
pivotdata.query("Finalizou == 'SIM'").sort_values('QuantidadeDiasRestantes')

Cliente,TotalRegistros,dia,TotalArquivosBanco,QuantidadeDiasRestantes,Finalizou
cau-al.implanta.net.br,2977,12/05/2021,33,0,SIM
crmv-to.implanta.net.br,5,12/05/2021,0,0,SIM
crn-ba.implanta.net.br,1774,12/05/2021,21,0,SIM
crn-df.implanta.net.br,2370,12/05/2021,23,0,SIM
crn-pa.implanta.net.br,119,12/05/2021,47,0,SIM
...,...,...,...,...,...
crea-ce.implanta.net.br,4015,12/05/2021,0,0,SIM
crea-es.implanta.net.br,2,12/05/2021,0,0,SIM
crea-ms.implanta.net.br,3,12/05/2021,0,0,SIM
crea-ac.implanta.net.br,169,12/05/2021,10,0,SIM


## Clientes em fase de migração

In [30]:
pivotdata.query("Finalizou == 'NAO'").sort_values('TotalArquivosBanco')


Cliente,TotalRegistros,dia,TotalArquivosBanco,QuantidadeDiasRestantes,Finalizou
cau-ba.implanta.net.br,1309,12/05/2021,52,0,NAO
crn-sc.implanta.net.br,1707,12/05/2021,52,0,NAO
core-ap.implanta.net.br,58,12/05/2021,58,0,NAO
cau-sc.implanta.net.br,3057,12/05/2021,62,0,NAO
cau-rn.implanta.net.br,3192,12/05/2021,64,0,NAO
...,...,...,...,...,...
crn-mg.implanta.net.br,13804,12/05/2021,12394,25,NAO
cau-mg.implanta.net.br,16036,12/05/2021,12945,26,NAO
cau-rs.implanta.net.br,18105,12/05/2021,14624,29,NAO
oab-rs.implanta.net.br,68104,12/05/2021,65692,131,NAO


# clientes migrados

In [31]:
transpose.query("Finalizou =='SIM'").shape

(154, 7)

# clientes ainda  não migrados

In [34]:
transpose.query("Finalizou =='NAO'").shape

(79, 7)

In [37]:
transpose.query("Finalizou =='NAO'").sort_values(['TotalArquivosBanco', 'TotalRegistros'], ascending=[True, False]).head(40)

,Cliente,TotalRegistros,TotalArquivosBanco,QuantidadeDeArquivosMigradosPorExecucao,dia,QuantidadeDiasRestantes,Finalizou
0,crn-sc.implanta.net.br,1707,52,500,12/05/2021,0,NAO
0,cau-ba.implanta.net.br,1309,52,500,12/05/2021,0,NAO
0,core-ap.implanta.net.br,58,58,500,12/05/2021,0,NAO
0,cau-sc.implanta.net.br,3057,62,500,12/05/2021,0,NAO
0,cau-rn.implanta.net.br,3192,64,500,12/05/2021,0,NAO
0,crf-pb.implanta.net.br,2485,65,500,12/05/2021,0,NAO
0,cau-go.implanta.net.br,4307,74,500,12/05/2021,0,NAO
0,crf-to.implanta.net.br,2504,78,500,12/05/2021,0,NAO
0,cau-ms.implanta.net.br,3824,79,500,12/05/2021,0,NAO
0,cau-pe.implanta.net.br,3451,83,500,12/05/2021,0,NAO


In [38]:
transpose.query("QuantidadeDiasRestantes in(1,2)")


,Cliente,TotalRegistros,TotalArquivosBanco,QuantidadeDeArquivosMigradosPorExecucao,dia,QuantidadeDiasRestantes,Finalizou
0,cau-ap.implanta.net.br,4078,346,500,12/05/2021,1,NAO
0,cau-es.implanta.net.br,3664,556,500,12/05/2021,1,NAO
0,cau-mt.implanta.net.br,4247,422,500,12/05/2021,1,NAO
0,cau-pb.implanta.net.br,4716,724,500,12/05/2021,1,NAO
0,cau-pi.implanta.net.br,3226,931,500,12/05/2021,2,NAO
0,cau-rr.implanta.net.br,3000,348,500,12/05/2021,1,NAO
0,cau-se.implanta.net.br,4387,400,500,12/05/2021,1,NAO
0,cau-to.implanta.net.br,4219,371,500,12/05/2021,1,NAO
0,crq-go.implanta.net.br,2474,537,500,12/05/2021,1,NAO
0,crq-pb.implanta.net.br,2741,371,500,12/05/2021,1,NAO


## inconsistencias

In [39]:
transpose.query("TotalRegistros == TotalArquivosBanco and TotalRegistros != 0")

,Cliente,TotalRegistros,TotalArquivosBanco,QuantidadeDeArquivosMigradosPorExecucao,dia,QuantidadeDiasRestantes,Finalizou
0,core-ap.implanta.net.br,58,58,500,12/05/2021,0,NAO
0,crea-rn.implanta.net.br,241,241,500,12/05/2021,0,NAO
0,crefito-ce.implanta.net.br,129,129,500,12/05/2021,0,NAO
0,crefito-df.implanta.net.br,477,477,500,12/05/2021,1,NAO
0,crefito-pa.implanta.net.br,48,48,500,12/05/2021,0,SIM
0,crefito-rj.implanta.net.br,356,356,500,12/05/2021,1,NAO
0,crp-ma.implanta.net.br,2,2,500,12/05/2021,0,SIM
0,oab-ap.implanta.net.br,1,1,500,12/05/2021,0,SIM
0,oab-rr.implanta.net.br,2,2,500,12/05/2021,0,SIM


## Output Dados in Excel

In [40]:
pivotdata.to_excel(pathFIleControleMigracao,sheet_name='registros')